In [1]:
from linear_models_functions import *

In [6]:
features = ['Score','White Percent', 'Black Percent', 'Hispanic Percent', 'High School Diploma or Higher', "Bachelor's Degree or higher", 'With an Advanced Degree', 'Percent Pop of US', 'Density', 'Average Income', 'Peak Unemployment', 'COVID Cases', 'COVID Deaths', 'Population', 'COVID Cases (% of Population)']
dates = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06']
dates2 = ['2020-05']#, '2020-04', '2020-05', '2020-06']

#run_models('ALCOHOL', dates2, plot=True)
#summarize_models_dem('DRINKING', waves, wave=True)
summarize_LASSO('ALCOHOL', dates2, features)

Optimal alpha:  9.085677934470695e-07
('Score', 8.872790904903126e-07)
('White Percent', -0.0)
('Black Percent', -1.4463412592595841e-06)
('Hispanic Percent', 1.3646610324457239e-05)
('High School Diploma or Higher', -8.595815797369943e-06)
("Bachelor's Degree or higher", 1.9099588762053445e-05)
('With an Advanced Degree', -1.3367760975380768e-05)
('Percent Pop of US', -0.0)
('Density', -6.365486417143341e-08)
('Average Income', -4.286982948903888e-09)
('Peak Unemployment', 6.978197810171601e-06)
('COVID Cases', 3.694589957887913e-10)
('COVID Deaths', -3.840126835295887e-10)
('Population', -3.529068272092341e-11)
('COVID Cases (% of Population)', -4.188951141596387e-07)


/home/johnattan/.local/share/virtualenvs/CHIP-GhDHLp-X/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.6380698345838177e-07, tolerance: 1.188165730419543e-10
  model = cd_fast.enet_coordinate_descent(
